# **  Classification of CIFAR-10 Dataset using Convolutional Neural Network**

---




1. Download the Dataset: If datafile is not already present download the dataset from [CIFAR-10 Dataset
](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz)





In [0]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
import tarfile

# Define the name of folder where dataset will be untared/unzipped
cifar10_dataset_folder_path = 'cifar-10-batches-py'

""" 
    check if the data (zip) file is already downloaded
    if not, download it from "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz" and save as cifar-10-python.tar.gz
"""
if not isfile('cifar-10-python.tar.gz'):
        print("Downloading CIFAR-10 Dataset.")
        urlretrieve(
            'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz',
            'cifar-10-python.tar.gz'
        )

# Checks the directory and extract the dataset.       
if not isdir(cifar10_dataset_folder_path):
    with tarfile.open('cifar-10-python.tar.gz') as tar:
        tar.extractall()
        tar.close()

# **Load CIFAR-10 dataset**

---

Define the functions for loading the dataset and label class names.

In [0]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

# Define CIFAR-10 label class names. There are 10 classes in the dataset as defined 
# below
def load_class_names():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

""" 
Loads the CIFAR-10 dataset into memory.

Params:
      cifar10_dataset_folder_path : Path of datset on local file system
      batch_id : Load's the dataset for given batch_id. This is done to load a 
      given part of dataset. The whole of the dataset can't be loaded into the 
      memory at the same time.
Returns:
      numpy arrays of dataset features and labels
"""  
def load_cfar10_for_batch(cifar10_dataset_folder_path, batch_id):
    with open(cifar10_dataset_folder_path + '/data_batch_' + str(batch_id), mode='rb') as file:
        # note the encoding type is 'latin1'
        batch = pickle.load(file, encoding='latin1')
        
    features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = batch['labels']
        
    return features, labels

# **Pre process the data**

---

Defines the functions for pre-processing data.

1. Normalize the features of CIFAR-10 dataset. Min-Max normalization is used.
2. One Hot Encoding is used for the Labels in the CIFAR-10 dataset. This is required for binarization of the categorical dense features/labels and include the encoded data for training


In [0]:
"""
Normalises the features in the Dataset. Min-Max normalization is used.

    Params:
        - features: input image data in numpy array [32, 32, 3]
    Return:
        - normalized features 
"""
def normalize(features):
    min_val = np.min(features)
    max_val = np.max(features)
    features = (features-min_val) / (max_val-min_val)
    return features

"""
One - Hot encoding of labels is used.   
      Params:
          - dense_labels: a list of dense labels
          - num_classes: Optional. Default 10 as per CIFAR-10 dataset.
      Returns:
          - one hot encoding matrix (number of labels, number of class)
"""    
def one_hot_encode(dense_labels, num_classes=10):
  encoded = np.zeros((len(dense_labels), num_classes))

  for idx, val in enumerate(dense_labels):
      encoded[idx][val] = 1

  return encoded

"""
Preprocess and save the dataset to a given path.
      Params:
          - normalize: normalised features of dataset.
          - one_hot_encode: one hot encoded labels of dataset
          - features: dense features from CIFAR-10 dataset.
          - labels: dense labesl from CIFAR-10 dataset.
          - filename: The path where pre-processed data is saved.
      
"""
def preprocess_and_save(normalize, one_hot_encode, features, labels, filename):
    features = normalize(features)
    labels = one_hot_encode(labels)

    pickle.dump((features, labels), open(filename, 'wb'))

"""
Preprocess and save the dataset to a given path.
      Params:
          - cifar10_dataset_folder_path: Path of CIFAR-10 folder on file system.
          - normalize: normalized features of dataset.
          - one_hot_encode: one hot encoded labels of dataset.
"""
def preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode):
    n_batches = 5
    valid_features = []
    valid_labels = []

    for batch_i in range(1, n_batches + 1):
        features, labels = load_cfar10_for_batch(cifar10_dataset_folder_path, batch_i)
        
        # validation data index in the whole dataset of the batch (5% of dataset)
        idx_val = int(len(features) * 0.05)

        # Preprocess the 95% of the dataset for training. rest 5% will be used 
        # for validation of the training.
        preprocess_and_save(normalize, one_hot_encode,
                             features[:-idx_val], labels[:-idx_val], 
                             'preprocess_batch_' + str(batch_i) + '.p')

        # 5% of batch dataset will be added to every batch as validation dataset 
        valid_features.extend(features[-idx_val:])
        valid_labels.extend(labels[-idx_val:])

    # preprocess validation dataset
    preprocess_and_save(normalize, one_hot_encode,
                         np.array(valid_features), np.array(valid_labels),
                         'preprocess_validation.p')

    # load test dataset
    with open(cifar10_dataset_folder_path + '/test_batch', mode='rb') as file:
        batch = pickle.load(file, encoding='latin1')

    # preprocess testing data
    test_features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    test_labels = batch['labels']

    # Preprocess and Save testing data
    preprocess_and_save(normalize, one_hot_encode,
                         np.array(test_features), np.array(test_labels),
                         'preprocess_training.p')
    
"""
Pre-process the data befor training
"""
preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode)

# **Convolution Neural Network Model**

---

The CNN model consists of 10 convolutional layers, 1 flattend layer of previous convolutional outputs and 5 fully connected layers. Breakdown of model parameters for each type of layers is defined as below:

keep_prob = 0.9

1.   Layer 1 & Layer 2
>  * Filter Size -> 3x3
>  * No. of Filters -> 32 
>  * Activation Function -> eLU
>  * Max Pooling ->  2
> * Dropout = 0 
>  * Batch Normalization

2.   Layer 3 & Layer 4
>  * Filter Size -> 3x3
>  * No. of Filters -> 64 
>  * Activation Function -> eLU
>  * Max Pooling ->  2
>  * Batch Normalization

3.   Layer 5 & Layer 6
>  * Filter Size -> 3x3
>  * No. of Filters -> 128 
>  * Activation Function -> eLU
>  * Max Pooling ->  2
>  * Batch Normalization

4.   Layer 7 & Layer 8
>  * Filter Size -> 3x3
>  * No. of Filters -> 256 
>  * Activation Function -> eLU
>  * Max Pooling ->  2
>  * Batch Normalization

5.   Layer 9
>  * Flattened output from last 8 Convolutional layers.

6.   Layer 10 
>  * Fully connected Layer
>  * Layer Units -> 64
>  * Dropout -> 0.1 (1 - keep_prob)
>  * Batch Normalization

7.   Layer 11 
>  * Fully connected Layer
>  * Layer Units -> 128
>  * Dropout -> 0.1 (1 - keep_prob)
>  * Batch Normalization

8.   Layer 12 
>  * Fully connected Layer
>  * Layer Units -> 256
>  * Dropout -> 0.1 (1 - keep_prob)
>  * Batch Normalization

9.   Layer 13 
>  * Fully connected Layer
>  * Layer Units -> 512
>  * Dropout -> 0.1 (1 - keep_prob)
>  * Batch Normalization

10.   Layer 14 
>  * Fully connected Layer
>  * Layer Units -> 10 (Equal to number of CIFAR-10 classes)
>  * Dropout -> 0
>  * Batch Normalization

In [0]:
import pickle
import tensorflow as tf

valid_features, valid_labels = pickle.load(open('preprocess_validation.p', mode='rb'))

# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
def reduce_var(x, axis=None, keepdims=False):
    """Variance of a tensor, alongside the specified axis.

    # Arguments
        x: A tensor or variable.
        axis: An integer, the axis to compute the variance.
        keepdims: A boolean, whether to keep the dimensions or not.
            If `keepdims` is `False`, the rank of the tensor is reduced
            by 1. If `keepdims` is `True`,
            the reduced dimension is retained with length 1.

    # Returns
        A tensor with the variance of elements of `x`.
    """
    m = tf.reduce_mean(x, axis=axis, keep_dims=True)
    devs_squared = tf.square(x - m)
    return tf.reduce_mean(devs_squared, axis=axis, keep_dims=keepdims)
  
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input_x')
y =  tf.placeholder(tf.float32, shape=(None, 10), name='output_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')


"""
Method that defines the CNN with 16 Layers. 
"""
def conv_net(x, keep_prob):
    conv1_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 3, 32], mean=0, stddev=0.05))
    conv2_filter = tf.Variable(tf.truncated_normal(shape=[3, 3, 32, 64], mean=0, stddev=0.05))
    conv3_filter = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 128], mean=0, stddev=0.05))
    conv4_filter = tf.Variable(tf.truncated_normal(shape=[5, 5, 128, 256], mean=0, stddev=0.05))

    # 1, 2
    conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,1,1,1], padding='SAME')
    conv1 = tf.nn.elu(conv1)
    conv1 = tf.nn.dropout(conv1, 1)
    conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv1_bn = tf.layers.batch_normalization(conv1_pool)

    # 3, 4
    conv2 = tf.nn.conv2d(conv1_bn, conv2_filter, strides=[1,1,1,1], padding='SAME')
    conv2 = tf.nn.elu(conv2)
    conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')    
    conv2_bn = tf.layers.batch_normalization(conv2_pool)
  
    # 5, 6
    conv3 = tf.nn.conv2d(conv2_bn, conv3_filter, strides=[1,1,1,1], padding='SAME')
    conv3 = tf.nn.elu(conv3)
    conv3_pool = tf.nn.max_pool(conv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')  
    conv3_bn = tf.layers.batch_normalization(conv3_pool)
    
    # 7, 8
    conv4 = tf.nn.conv2d(conv3_bn, conv4_filter, strides=[1,1,1,1], padding='SAME')
    conv4 = tf.nn.elu(conv4)
    conv4_pool = tf.nn.max_pool(conv4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    conv4_bn = tf.layers.batch_normalization(conv4_pool)
        
    # 9
    flat = tf.contrib.layers.flatten(conv4_bn)  

    # 10
    full1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=34, activation_fn=tf.nn.elu)
    full1 = tf.nn.dropout(full1, keep_prob)
    full1 = tf.layers.batch_normalization(full1)
    
    # 11
    full2 = tf.contrib.layers.fully_connected(inputs=full1, num_outputs=64, activation_fn=tf.nn.elu)
    full2 = tf.nn.dropout(full2, keep_prob)
    full2 = tf.layers.batch_normalization(full2)
    
    # 12
    full3 = tf.contrib.layers.fully_connected(inputs=full2, num_outputs=128, activation_fn=tf.nn.elu)
    full3 = tf.nn.dropout(full3, keep_prob)
    full3 = tf.layers.batch_normalization(full3)    
    
    # 13
    full4 = tf.contrib.layers.fully_connected(inputs=full3, num_outputs=256, activation_fn=tf.nn.elu)
    full4 = tf.nn.dropout(full4, keep_prob)
    full4 = tf.layers.batch_normalization(full4)        
    
    # 14
    out = tf.contrib.layers.fully_connected(inputs=full4, num_outputs=10, activation_fn=None)
    out = tf.nn.dropout(out, 1)
    return out

> # **Training of Model**

In [47]:
epochs = 20
batch_size = 64
keep_probab = 0.9 
learning_rate = 1e-3

# logits tensor so that it can loaded from disk after training
logits = conv_net(x, keep_probab)
model = tf.identity(logits, name='logits') 

# Loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# Stochastic gradient descent optimizer using Adaptive Moment (ADAM) Estimation 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

def train_neural_network(session, optimizer, keep_probab, feature_batch, label_batch):
    session.run(optimizer, 
                feed_dict={
                    x: feature_batch,
                    y: label_batch,
                    keep_prob: keep_probab
                })

def print_stats(session, feature_batch, label_batch, cost, accuracy):
    loss = sess.run(cost, 
                    feed_dict={
                        x: feature_batch,
                        y: label_batch,
                        keep_prob: 1.
                    })
    valid_acc = sess.run(accuracy, 
                         feed_dict={
                             x: valid_features,
                             y: valid_labels,
                             keep_prob: 1.
                         })
    
    print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss, valid_acc))
    
"""
Feature & Labels are split in batches
"""    
def batch_features_labels(features, labels, batch_size):
    for start in range(0, len(features), batch_size):
        end = min(start + batch_size, len(features))
        yield features[start:end], labels[start:end]

"""
Loads the preprocessed training data and return's loaded data in batches
"""
def load_preprocess_training_batch(batch_id, batch_size):
    filename = 'preprocess_batch_' + str(batch_id) + '.p'
    features, labels = pickle.load(open(filename, mode='rb'))
    return batch_features_labels(features, labels, batch_size)
  
save_model_path = './image_classification'

print('Model being trained....')
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Operations performed in a training cycle iteratively.
    for epoch in range(epochs):
        # Loop over all batches
        n_batches = 5
        for batch_i in range(1, n_batches + 1):
            for batch_features, batch_labels in load_preprocess_training_batch(batch_i, batch_size):
                train_neural_network(sess, optimizer, keep_probab, batch_features, batch_labels)
                
            print('Epoch {:>2}, Batch# {}:  '.format(epoch + 1, batch_i), end='')
            print_stats(sess, batch_features, batch_labels, cost, accuracy)
            
    # After certain epochs, 
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)

Model being trained....
Epoch  1, Batch# 1:  Loss:     1.2946 Validation Accuracy: 0.435600
Epoch  1, Batch# 2:  Loss:     1.0340 Validation Accuracy: 0.525600
Epoch  1, Batch# 3:  Loss:     0.9909 Validation Accuracy: 0.532000
Epoch  1, Batch# 4:  Loss:     1.1525 Validation Accuracy: 0.579200
Epoch  1, Batch# 5:  Loss:     0.7760 Validation Accuracy: 0.604400
Epoch  2, Batch# 1:  Loss:     0.5799 Validation Accuracy: 0.626400
Epoch  2, Batch# 2:  Loss:     0.6705 Validation Accuracy: 0.647600
Epoch  2, Batch# 3:  Loss:     0.6941 Validation Accuracy: 0.648000
Epoch  2, Batch# 4:  Loss:     0.5594 Validation Accuracy: 0.687200
Epoch  2, Batch# 5:  Loss:     0.6200 Validation Accuracy: 0.664400
Epoch  3, Batch# 1:  Loss:     0.3215 Validation Accuracy: 0.678000
Epoch  3, Batch# 2:  Loss:     0.5950 Validation Accuracy: 0.668000
Epoch  3, Batch# 3:  Loss:     0.4068 Validation Accuracy: 0.687600
Epoch  3, Batch# 4:  Loss:     0.3474 Validation Accuracy: 0.689200
Epoch  3, Batch# 5:  Los

In [49]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random

def batch_features_labels(features, labels, batch_size):
    for start in range(0, len(features), batch_size):
        end = min(start + batch_size, len(features))
        yield features[start:end], labels[start:end]

save_model_path = './image_classification'
batch_size = 512
n_samples = 10
top_n_predictions = 5

def test_model():
    test_features, test_labels = pickle.load(open('preprocess_training.p', mode='rb'))
    loaded_graph = tf.Graph()

    with tf.Session(graph=loaded_graph) as sess:
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(sess, save_model_path)

        loaded_x = loaded_graph.get_tensor_by_name('input_x:0')
        loaded_y = loaded_graph.get_tensor_by_name('output_y:0')
        loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
        loaded_logits = loaded_graph.get_tensor_by_name('logits:0')
        loaded_acc = loaded_graph.get_tensor_by_name('accuracy:0')
        
        test_batch_acc_total = 0
        test_batch_count = 0
        
        for train_feature_batch, train_label_batch in batch_features_labels(test_features, test_labels, batch_size):
            test_batch_acc_total += sess.run(
                loaded_acc,
                feed_dict={loaded_x: train_feature_batch, loaded_y: train_label_batch, loaded_keep_prob: 1.0})
            test_batch_count += 1

        print('Testing Accuracy: {}\n'.format(test_batch_acc_total/test_batch_count))
test_model()

INFO:tensorflow:Restoring parameters from ./image_classification
Testing Accuracy: 0.7048540890216828

